<h1>Exercise 4: The Multi-Step Calculator</h1> 

### The Challenge
Build a complex calculator that performs two steps of calculations sequentially, with decisions at each step.

**Requirements:**
1. **Step 1**: Check `operator1`.
    - If `+`, add `num1` and `num2`.
    - If `-`, subtract `num2` from `num1`.
    - Store result in `finalNum`.
2. **Step 2**: Check `operator2`. 
    - If `+`, add `num3` and `num4`.
    - If `-`, subtract `num4` from `num3`.
    - Store result in `finalNum2`.
3. **Flow**: Step 2 should happen *after* Step 1 is done.

### The Solution
This is a bit tricky, but we can do it! We just need to chain our conditional logic.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

### Step 1: State
We need a lot of variables here! Inputs for both operations and outputs for both.

In [ ]:
class AgentState(TypedDict):
    num1: int
    operator1: str
    num2: int
    num3: int
    num4: int
    operator2: str
    finalNum: int
    finalNum2: int

### Step 2: The Logic nodes (Round 1)
First, we handle `num1` and `num2`.

In [ ]:
def addNode(state: AgentState) -> AgentState:
    """Add num1 and num2 and store the result in finalNum"""
    state["finalNum"] = state["num1"] + state["num2"]
    return state

In [ ]:
def subNode(state: AgentState) -> AgentState:
    """Subtract num2 from num1 and store the result in finalNum"""
    state["finalNum"] = state["num1"] - state["num2"]
    return state

In [ ]:
def routerFunc1(state: AgentState) -> str:
    """This is the conditional logic for first router"""
    if(state["operator1"] == "+"):
        return "addEdge"
    elif(state["operator1"] == "-"):
        return "subEdge"

### Step 2: The Logic nodes (Round 2)
Now we handle `num3` and `num4`.

In [ ]:
def addNode2(state: AgentState) -> AgentState:
    """Adds num3 and num4 and stores in finalNum2"""
    state["finalNum2"] = state["num3"] + state["num4"]
    return state

In [ ]:
def subNode2(state: AgentState) -> AgentState:
    """Subtracts num4 from num3 and stores in finalNum2"""
    state["finalNum2"] = state["num3"] - state["num4"]
    return state
    

In [ ]:
def routerFunc2(state: AgentState) -> str:
    """Conditional logic for router 2"""
    if(state["operator2"] == "+"):
        return "addEdge2"
    elif(state["operator2"] == "-"):
        return "subEdge2"

### Step 3: The Complex Graph
We chain the two sets of operations.
1. Start -> Router1
2. Router1 decices: Go to Adder1 or Sub1
3. Adder1 and Sub1 *both* point to Router2. This is key! It means "after I'm done, go to the next decision point".
4. Router2 decides: Go to Adder2 or Sub2
5. Both end at the finish line.

In [ ]:
graph = StateGraph(AgentState)

# Add all nodes
graph.add_node("adder", addNode)
graph.add_node("sub", subNode)
graph.add_node("adder2", addNode2)
graph.add_node("sub2", subNode2)
graph.add_node("router1", lambda x: x) #passthrough node
graph.add_node("router2", lambda x: x) #passthrough node

# Start -> Router 1
graph.add_edge(START, "router1")

# Router 1 Logic
graph.add_conditional_edges(
    "router1", 
    routerFunc1, 
    {
        "addEdge": "adder",
        "subEdge": "sub"
    }
)

# Connect end of Step 1 to Router 2
graph.add_edge("adder", "router2")
graph.add_edge("sub", "router2")

# Router 2 Logic
graph.add_conditional_edges(
    "router2", 
    routerFunc2,
    {
        "addEdge2": "adder2",
        "subEdge2": "sub2"
    }
)

# Finish
graph.add_edge("adder2", END)
graph.add_edge("sub2", END)

app = graph.compile()

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({
    "num1": 10, "operator1": "-", "num2": 5, 
    "num3": 7, "num4": 2, "operator2": "+", 
    "finalNum": 0, "finalNum2": 0
})
print(result)